In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

In [18]:
df_tfidf = pd.read_csv(r"../gerador-tf-idf/tfidf.csv", sep=",")
df_docs_classificados = pd.read_csv(r"../extracao-textos/documentos-classificados.csv", sep=",")
df_docs_classificados['categoriaNumero'] = df_docs_classificados['categoria'].map({
        'contrato': 1,
        'licitacao': 2,
        'notaempenho': 3,
        'sancao': 4
    })

In [ ]:
################## MODELO DE CLASSIFICAÇÃO NAIVE BAYES (MultinomialNB) ##################

# Dividir o dataframe em recursos (X) e alvo (y)
X_recursos = df_tfidf
y_alvo = df_docs_classificados['categoriaNumero']

X_train, X_test, y_train, y_test = train_test_split(X_recursos, y_alvo, test_size=0.5, random_state=0)

# Aplicar SMOTE (apenas aos dados de treinamento)
smote = SMOTE(random_state=0)
X_train_balanceado, y_train_balanceado = smote.fit_resample(X_train, y_train)

# Trainar modelo e prever
mnb = MultinomialNB()
mnb.fit(X_train_balanceado, y_train_balanceado)
y_pred = mnb.predict(X_test)

# Exibir um relatório de classificação
print(classification_report(y_test, y_pred))
accuracy = accuracy_score(y_test, y_pred)
print("Precisão do Multinomial Naive Bayes: {:.2f}%".format(accuracy * 100))
print("De um total de %d tweets, foram rotulados incorretamente: %d"
      % (X_test.shape[0], (y_test != y_pred).sum()))

              precision    recall  f1-score   support

           1       0.93      0.98      0.96        57
           2       1.00      1.00      1.00        85
           3       1.00      0.96      0.98        93
           4       0.93      1.00      0.97        14

    accuracy                           0.98       249
   macro avg       0.97      0.98      0.98       249
weighted avg       0.98      0.98      0.98       249

Precisão do Multinomial Naive Bayes: 97.99%
De um total de 249 tweets, foram rotulados incorretamente: 5


In [20]:
# Salvar modelo com Joblib #
joblib.dump(mnb, r'../API-modeloML/modeloFinal.pkl')
print("Modelo salvo com sucesso!")

Modelo salvo com sucesso!
